In [1]:
include("../Engine/Engine.jl")
using .Engine

using StatsBase, DataFrames
using Graphs, SimpleWeightedGraphs
import PyPlot as plt
import Seaborn as sns
import PlotlyJS

rcParams = plt.PyDict(plt.matplotlib."rcParams")
rcParams["text.usetex"] = false

WebIO._IJuliaInit()

false

In [2]:
cop26 = load_dataset(COP26)
ranks = get_all_ranks(cop26, cop_26_dates)

graphs, cascades, df = load_data("/home/ubuntu/Thesis/Results/JDD_all_users/COP26/data.jld2")
graphs2, cascades2, df2 = load_data("/home/ubuntu/Thesis/Results/JDD_all_users/Random/data.jld2");

In [ ]:
ranks = get_all_ranks(COP26, cop_26_dates)

In [ ]:
ranks2 = get_centrality_ranks(graphs, df, 0, "Any Edge")

In [ ]:
function correlation_matrices(ranks, centrality_ranks, N = 50)

    if length(ranks) != length(centrality_ranks)
        throw(ArgumentError("Number of partition mismatch."))
    end

    output = []
    labels = []

    for (r1, r2) in zip(ranks, centrality_ranks)
        partition = r1.partition[1]
        if r1.username != r2.username
            throw(ArgumentError("Both do not contain the same usernames."))
        end
        tot = innerjoin(r1, r2, on=:username)

        cols = [name for name in names(tot) if occursin("rank", name)]
        labels_ = [split(col, '_')[1] for col in cols]
        corr_matrix = zeros(length(cols), length(cols))

        for (i, col1) in enumerate(cols)
            sorting = sortperm(tot[!, col1])
            users1 = tot.username[sorting][1:N]
            # Should be 1:N
            rank1 = tot[!, col1][sorting][1:N]
            if rank1 != collect(1:N)
                throw(ArgumentError("error"))
            end
            
            for (j, col2) in enumerate(cols)

                # indices = [findfirst(user .== tot.username) for user in users1]
                # users2 = tot.username[indices]
                # rank2 = tot[!, col2][indices]
                # corr_matrix[i,j] = corspearman(rank1, rank2)
                sorting = sortperm(tot[!, col2])
                users2 = tot.username[sorting][1:N]
                corr_matrix[i,j] = length(intersect(users1, users2)) / length(users1)

            end
        end

        push!(output, corr_matrix)
        push!(labels, labels_)

    end

    return output, labels

end

out, labels = correlation_matrices(ranks, ranks2, 50)

plt.figure()
sns.heatmap(out[3], annot=true, cmap="rocket")
xloc, xlabels = plt.xticks()
plt.xticks(xloc, labels[1], rotation="vertical")
yloc, ylabels = plt.yticks()
plt.yticks(yloc, labels[1], rotation="horizontal")
# plt.savefig("/home/ubuntu/Thesis/Figures/JDD_all_users/COP26/heatmap3.pdf", bbox_inches="tight")
plt.gcf()


In [ ]:
edge_type = "Any Edge"
N_biggest = 20
cuttoff = 0

between, between_actors = betweenness_centralities(graphs, df, cuttoff=cuttoff, edge_type=edge_type)
in_, in_actors = indegree_centralities(graphs, df, cuttoff=cuttoff, edge_type=edge_type)
out, out_actors = outdegree_centralities(graphs, df, cuttoff=cuttoff, edge_type=edge_type)

between_actors[3][1:N_biggest]

In [ ]:
plot_edge_types([graphs, graphs2], [df, df2], [0,0]; y="count_normalized", hue_order=["Before COP26", "During COP26", "After COP26", "Control"],
 save=true, filename="/home/ubuntu/Thesis/Figures/JDD_all_users/COP26/edge_type.pdf", log=true)

In [ ]:
plot_actors_per_level(cascades, df, control=cascades2, save=true, filename="/home/ubuntu/Thesis/Figures/JDD_all_users/COP26/actor_level.pdf", inner_spacing=0)

In [36]:
cascades[3]
PlotlyJS.plot(plot_cascade_sankey(cascades[3][5], df)...)
# PlotlyJS.savefig(PlotlyJS.plot(plot_cascade_sankey(cascades[3][5], df)...), "/home/ubuntu/Thesis/Figures/JDD_all_users/COP26/cascade.pdf")

"/home/ubuntu/Thesis/Figures/JDD_all_users/COP26/cascade.pdf"